<div class="notebook-buttons" style="display:flex; padding-top: 5rem;padding-bottom: 2.5rem;line-height: 2.15;">
    <a href="https://colab.research.google.com/github/zazuko/notebooks/blob/master/notebooks/environment/heavy_metal.ipynb">
        <div id="colab-link" style="display: flex;padding-right: 3.5rem;padding-bottom: 0.625rem;border-bottom: 1px solid #ececed; align-items: center;">
            <img class="call-to-action-img" src="img//colab.svg" width="30" height="30" style="margin-right: 10px;margin-top: auto;margin-bottom: auto;">
            <div class="call-to-action-txt">Run in Google Colab</div>
        </div>
    </a>
    <a href="https://raw.githubusercontent.com/zazuko/notebooks/master/notebooks/environment/heavy_metal.ipynb" download>
        <div id="download-link" style="display: flex;padding-right: 3.5rem;padding-bottom: 0.625rem;border-bottom: 1px solid #ececed; height: auto;align-items: center;">
            <img class="call-to-action-img" src="img//download.svg" width="22" height="30" style="margin-right: 10px;margin-top: auto;margin-bottom: auto;">
            <div class="call-to-action-txt">Download Notebook</div>
        </div>
    </a>
    <a href="https://github.com/zazuko/notebooks/blob/master/notebooks/environment/heavy_metal.ipynb">
        <div id="github-link" style="display: flex;padding-right: 3.5rem;padding-bottom: 0.625rem;border-bottom: 1px solid #ececed; height: auto;align-items: center;">
            <img class="call-to-action-img" src="img//github.svg" width="25" height="30" style="margin-right: 10px;margin-top: auto;margin-bottom: auto;">
            <div class="call-to-action-txt">View on GitHub</div>
        </div>
    </a>
</div>

# Heavy metal soil pollution in Switzerland

FSVO, Federal Food Safety and Veterinary Office, collects data on the animal diseases in Switzerland. This data is published as [Linked Data](https://en.wikipedia.org/wiki/Linked_data). 

In this tutorial, we will show **how to work with Linked Data.** Mainly, we will see how to work with data on animal disease.   
We will look into how to query, process, and visualize it.   

[1. Setup](#Setup)    
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[1.1 Python](#SPARQL-endpoints)    
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[1.2 SPARQL endpoints](#SPARQL-endpoints)    
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[1.3 SPARQL client](#SPARQL-endpoints)    

[2. Data](#Data)        
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[2.1 Animal species](#Animal-species)    
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[2.2 Diseases](#Diseases)    
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[2.3 Can we link disease to animal type?](#Can-we-link-disease-to-animal-type)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[2.4 Reports](#Reports)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[2.5 Example: goats](#example-goats)  


[3. Analysis](#Analysis)    
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[3.1 Cattle: Mucosal Disease](#cattle-mucosal-disease)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[3.2 Bees: Sauerbrut](#Sauerbrut)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[3.3 Rabies](#Rabies)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[3.4 Most common diseases](#Most-common-diseases)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[3.5 Reports over time](#reports-over-time)  



## Setup

### Python
This notebook requires Python 3.9 or later to run.

### SPARQL endpoints

#### For epidemiological data
Reports on animal diseases are published as Linked Data. It can be accessed with [SPARQL queries](https://www.w3.org/TR/rdf-sparql-query/).   
You can send queries using HTTP requests. The API endpoint is **[https://lindas.admin.ch/query/](https://int.lindas.admin.ch/query).**  

#### For geodata
Animal diseases are closely linked to places. To understand their location, we will work with 
Swiss geodata. It is published as Linked Data. It can be accessed using API endpoint under **[https://ld.geo.admin.ch/query](https://ld.geo.admin.ch/query).**  

### SPARQL client

Let's use `SparqlClient` from [graphly](https://github.com/zazuko/graphly) to communicate with both databases. 
Graphly will allow us to:
* send SPARQL queries
* automatically add prefixes to all queries
* format response to `pandas` or `geopandas`

In [ ]:
import datetime
import itertools
import json

import folium
import mapclassify
import matplotlib as mpl
import matplotlib.cm
import networkx as nx
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from bokeh.io import output_notebook, show
from bokeh.models import (Circle, CustomJS, HoverTool, LabelSet, MultiLine,
                          NodesAndLinkedEdges, Plot, TapTool)
from bokeh.palettes import Paired
from bokeh.plotting import ColumnDataSource, from_networkx
from dateutil.relativedelta import relativedelta
from folium.plugins import TimeSliderChoropleth
from graphly.api_client import SparqlClient
from plotly.subplots import make_subplots
from enum import Enum
from collections import namedtuple

In [ ]:
# Uncomment to install dependencies in Colab environment
#!pip install mapclassify
#!pip install git+https://github.com/zazuko/graphly.git

In [ ]:
sparql = SparqlClient("https://lindas.admin.ch/query")
geosparql = SparqlClient("https://ld.geo.admin.ch/query")

sparql.add_prefixes({
    "schema": "<http://schema.org/>",
    "cube": "<https://cube.link/>",
    "admin": "<https://schema.ld.admin.ch/>",
    "skos": "<http://www.w3.org/2004/02/skos/core#>",
    "disease": "<https://environment.ld.admin.ch/foen/animal-pest/>"   
})

geosparql.add_prefixes({
    "dct": "<http://purl.org/dc/terms/>",
    "geonames": "<http://www.geonames.org/ontology#>",
    "schema": "<http://schema.org/>",
    "geosparql": "<http://www.opengis.net/ont/geosparql#>",
})

SPARQL queries can become very long. To improve the readibility, we will work wih [prefixes](https://en.wikibooks.org/wiki/SPARQL/Prefixes).
 
Using the `add_prefixes` method, we can define persistent prefixes. 
Every time you send a query, `graphly` will now automatically add the prefixes for you.

Heavy metals (list): https://s.zazuko.com/4kTuJ6

https://ld.admin.ch/cube/dimension/el01/cd -> max 5 obs per station
https://ld.admin.ch/cube/dimension/el01/co -> max 6 obs
https://ld.admin.ch/cube/dimension/el01/cu -> max 6 obs
https://ld.admin.ch/cube/dimension/el01/hg -> max 6 obs
https://ld.admin.ch/cube/dimension/el01/ni -> max 6 obs
https://ld.admin.ch/cube/dimension/el01/pb -> max 6 obs
https://ld.admin.ch/cube/dimension/el01/zn -> max 6 obs

Measurement stations + land use: https://s.zazuko.com/5gBv2u

Metal + station + year:
- different amount of stations every year (3-43)



Dimension:
 - station (geo) + land use
 - metal (cat)
 - time

Ideas:
* map with ALL measurement stations
* ranges for all metals over ALL times
* distribution of measurements (aggregate time and space, choose one metal). Be smart to not oversample one item!
* distribution of measurements (aggregate time and space, choose one metal + one land use)
* distribution over time -> are we getting better ?


* time seires: value + time (one metal, one place)
* map: value + place (one metal, one year)
* barplot: value + metal (one year, one place)

aggregations:
 - average metal concentration in (land use) and (year) -> for each year